<a href="https://colab.research.google.com/github/SutirthaChakraborty/MatchMiDi/blob/main/MIDIsignatureV3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!rm -rf data
!rm -rf MixcraftDataset
!rm -rf testingdatapaper/
!rm -rf testingFiles/
!rm -rf dataFiles/

!git clone https://github.com/SutirthaChakraborty/MatchMiDi.git



!pip install py_midicsv
!pip install python-Levenshtein
!unzip /content/MatchMiDi/data.zip
!unzip /content/MatchMiDi/MixcraftDataset.zip
!pip install progressbar
from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')




In [76]:
import Levenshtein
def distance(tar,total):
    k=[]
    for i in tqdm(range(len(total)-len(tar))):
        k.append(Levenshtein.distance(str(tar),str(total[i:i+len(tar)])))
    # plt.plot(k)
    block=k.index(min(k))
    # print("Minimum value= ",min(k))
    # print("time =", total['Timestamp'].iloc[block+1])
    return k,block


In [3]:
import py_midicsv as pm
import pandas as pd
import numpy as np
import glob
total=pd.DataFrame()

dataSet=pd.DataFrame()



t=1
files=[]
timediff=[]
pitchData=[]


for file in glob.glob("/content/data/*.mid"):
    files.append(file)
files=sorted(files)


for file in tqdm(files):
    csv_string1 = pm.midi_to_csv(file)
    content = [x.strip().split(', ') for x in csv_string1] 
    data=pd.DataFrame(content)
    data.columns=['A','Timestamp','NotePress','Track','Pitch','Velocity','G']
    y=pd.DataFrame([t] * data.shape[0])
    y.columns=['FileID']
    data=pd.concat([data,y],axis=1)
    total=total.append(data, ignore_index=True)
    t+=1
    total=total[15:-2]
    a=list(total.Track.unique())
    a=filter(None, a) # Remove nulls
    target_Tracks = [ i for i in a if i.isnumeric() ]    # Calculates the number of tracks
    for i in target_Tracks:


        tar=total.loc[total['Track'] == i]
        tar=tar.loc[(tar['NotePress'] == 'Note_off_c') | (tar['NotePress'] == 'Note_on_c') ]
    
        dataSet=dataSet.append(tar, ignore_index=True)
        if(int(i)>-1 and int(i)<=5):
            name= str(i)+".csv"
            tar.to_csv(name)

        if(len(list(np.diff(np.array(tar['Timestamp']).astype(float))))>0):
            pidata=list(tar['Pitch'].astype(float))
            pitchData=pitchData+pidata
            xx=[0]+list(np.diff(np.array(tar['Timestamp']).astype(float)))
            b = (xx - np.min(xx))/np.ptp(xx)
            timediff=timediff+list(b)


len(timediff),len(pitchData)

d=pd.DataFrame(timediff)
d.columns=['TimeDiff']
d.to_csv('timediff.csv')

d=pd.DataFrame(pitchData)
d.columns=['pitchData']
d.to_csv('pitchData.csv')


In [130]:
target_path='/content/MixcraftDataset/File - 03 - 6 sec.mid'

import py_midicsv as pm
import pandas as pd
import numpy as np
import glob

tar_timediff=[]
tar_pitchData=[]

timestamp_collector=[]
file_collector=[]
track_collector=[]

blocks=[]


csv_string1 = pm.midi_to_csv(target_path)
content = [x.strip().split(', ') for x in csv_string1] 
data=pd.DataFrame(content)

data.columns=['A','Timestamp','NotePress','Track','Pitch','Velocity','G']
total=data

t+=1
total=total[15:-2]
a=list(total.Track.unique())
a=filter(None, a) # Remove nulls
target_Tracks = [ i for i in a if i.isnumeric() ]    # Calculates the number of tracks
for i in (target_Tracks):

    tar=total.loc[total['Track'] == i]
    tar=tar.loc[(tar['NotePress'] == 'Note_off_c') | (tar['NotePress'] == 'Note_on_c') ]
    filename=str(i)+'find.csv'
    tar.to_csv(filename)

    if(len(list(np.diff(np.array(tar['Timestamp']).astype(float))))>0):
        pidata=list(tar['Pitch'].astype(float))
        tar_pitchData=tar_pitchData+pidata

        xx=[0]+list(np.diff(np.array(tar['Timestamp']).astype(float)))
        b = (xx - np.min(xx))/np.ptp(xx)
        tar_timediff=tar_timediff+list(b)

        filename = str(i)+"time.csv"
        d=pd.DataFrame(tar_timediff)
        d.columns=['tar_timediff']
        d.to_csv(filename)

        filename = str(i)+"pitch.csv"
        d=pd.DataFrame(tar_pitchData)
        d.columns=['pitchData']
        d.to_csv(filename)

        targetfile=str(i)+".csv"
        datasetfile=pd.read_csv(targetfile)

        k,block=distance(pidata,list(datasetfile.Pitch))
        # print(block)
        # b = (k - np.min(k))/np.ptp(k)
        # plt.plot(b)
        # plt.show()
        file_collector.append(datasetfile['FileID'].iloc[block])
        timestamp_collector.append(datasetfile['Timestamp'].iloc[block])
        track_collector.append(datasetfile['Track'].iloc[block])
        blocks.append(block)



# len(tar_timediff),len(tar_pitchData)
track_collector,timestamp_collector,file_collector

([0, 1], [28800, 28800], [3, 3])

In [120]:
def closest(lst, K): 
	return lst[min(range(len(lst)), key = lambda i: abs(lst[i]-K))] 


def most_frequent(List): 
	return max(set(List), key = List.count) 

confirmedfile=most_frequent(file_collector)
print("File : ",most_frequent(file_collector))

print("Time : ",most_frequent(timestamp_collector)) 

	

File :  3
Time :  28800


In [121]:
all_occurrences = []
last_found_index = -1
element_found = True
timelist=[]

# finding out most occured file to confirm the file number

while element_found:
    try:
        last_found_index = file_collector.index(most_frequent(file_collector), last_found_index + 1)
        all_occurrences.append(last_found_index)
    except ValueError:
        element_found = False


# Finding the most likely time to confirm the probable time

for i in range(len(timestamp_collector)):
    if i in all_occurrences:
        timelist.append(timestamp_collector[i])
likely_time=most_frequent(timelist)
print("Most Likely time = ",likely_time )


K = likely_time

# Search the closest midi point from that time  

timestamps=[]
for i in target_Tracks:
    filename=i+'.csv'
    temp=pd.read_csv(filename)
    temp=list(temp.Timestamp)
    timestamps.append(closest(temp,likely_time))



Most Likely time =  28800


**Correct Value Should be around 20**

We chopped all the MIDI from around 20 secs

In [122]:
# print(60 * 1000000 / 100)  # 100 was the bpm when we edited the dataset
print(600000/likely_time)
print(confirmedfile)

20.833333333333332
3


In [176]:
likely_time

28800

**Getting the timestretch**

In [183]:
file1=pd.read_csv('/content/0.csv')
file2=pd.read_csv('0find.csv')

temp=file1[file1.Timestamp==likely_time]
temp[temp.FileID==confirmedfile]
l=temp[temp.FileID==confirmedfile]
loc=l['Unnamed: 0']
# print(loc)

a=float(600000/int(file1.Timestamp.iloc[loc+1-15]))
b=float(600000/int(file1.Timestamp.iloc[loc+2-15]))

print(b-a)
c=float(600000/int(file2.Timestamp.iloc[1]))
d=float(600000/int(file2.Timestamp.iloc[2]))

print(d-c)

print(600000/int(file1.Timestamp.iloc[(loc+1)-15]))

print(600000/int(file1.Timestamp.iloc[(loc+2)-15]))

# print(file1.iloc[loc+1-15])
# print(file1.iloc[loc+2-15])
# print(file1.iloc[loc+3-15])
# # # file2.iloc[0]
# file2

-1.552409339294588
-1000.0
20.491803278688526
18.939393939393938


In [191]:
file1.iloc[loc+1]

,Unnamed: 0,A,Timestamp,NotePress,Track,Pitch,Velocity,G,FileID
26,41,2,36960,Note_on_c,0,77,81,NaN,3
